In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Sequential, load_model
from keras.layers import Input, BatchNormalization, Activation, GlobalAveragePooling2D, UpSampling2D
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, fbeta_score

import seaborn as sns

import pandas as pd
import numpy as np
import cv2
from matplotlib import pyplot as plt

def prepare_dataset(dataframe, image_size={'width':48, 'height':48}):
    faces_images = []
    pixels = dataframe['pixels'].tolist()
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(image_size['width'], image_size['height'])
        face = cv2.resize(face.astype('uint8'), (image_size['width'], image_size['height']))        
        faces_images.append(face.astype('float32') / 255.0)
    faces_images = np.asarray(faces_images)
    faces_images = np.expand_dims(faces_images, -1) # (1, 48, 48)
    emotions_labels = pd.get_dummies(dataframe['emotion']).as_matrix()
    return faces_images, emotions_labels
  
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 1

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin), axis=1) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)), axis=1)
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)), axis=1)

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return K.mean((beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon()))

In [ ]:
dataset = pd.read_csv('../input/fer2013.csv')

In [ ]:
emotions_labels_map = {'angry':0,
                       'disgust':1, 
                       'fear':2, 
                       'happy':3, 
                       'sad':4, 
                       'surprise':5, 
                       'neutral':6}

groups = dataset.groupby('Usage')

# get group - training
training_data = groups.get_group('Training')

# get groups on training split by emotion
training_emotions = training_data.groupby('emotion')

# get group - test
test_data = groups.get_group('PublicTest')

# get groups on test split by emotion
test_emotions = test_data.groupby('emotion')

# get group - validation
validation_data = groups.get_group('PrivateTest')

# get groups on validation split by emotion
validation_emotions = validation_data.groupby('emotion')

# prepare dataset to training, test and validate model
training_faces, training_emotions = prepare_dataset(training_data)
test_faces, test_emotions = prepare_dataset(test_data)
validation_faces, validation_emotions = prepare_dataset(validation_data)

# get size dataset elements
num_samples, num_classes = training_emotions.shape

# split and randomize data
x_train_part1, x_train_part2, y_train_part1, y_train_part2 = train_test_split(training_faces, training_emotions, test_size=0.3, random_state=42)
x_train, y_train = np.vstack((x_train_part1, x_train_part2)), np.vstack((y_train_part1, y_train_part2))

x_test_part1, x_test_part2, y_test_part1, y_test_part2 = train_test_split(test_faces, test_emotions, test_size=0.3, random_state=42)
x_test, y_test = np.vstack((x_test_part1, x_test_part2)), np.vstack((y_test_part1, y_test_part2))

x_validation_part1, x_validation_part2, y_validation_part1, y_validation_part2 = train_test_split(validation_faces, validation_emotions, test_size=0.3, random_state=42)
x_validation, y_validation = np.vstack((x_validation_part1, x_validation_part2)), np.vstack((y_validation_part1, y_validation_part2))

In [ ]:
# create data augmention
data_generator = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images
data_generator.fit(x_train)
data_generator.fit(x_test)
data_generator.fit(x_validation)

In [ ]:
patience = 50
batch_size = 86
num_epochs = 100

early_stop = EarlyStopping(monitor='val_acc', patience=patience, mode='max')
reduce_lr = ReduceLROnPlateau('val_acc', factor=0.5, patience=(patience//4))
callbacks = [early_stop, reduce_lr]

In [ ]:
model = Sequential()
model.add(Convolution2D(64, (3, 3), kernel_initializer='he_normal', activation='relu', padding='same', input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Convolution2D(64, (3, 3), kernel_initializer='he_normal', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.5))

model.add(Convolution2D(128, (3, 3), kernel_initializer='he_normal', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Convolution2D(128, (3, 3), kernel_initializer='he_normal', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.5))

model.add(Convolution2D(256, (3, 3), kernel_initializer='he_normal', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Convolution2D(256, (3, 3), kernel_initializer='he_normal', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Convolution2D(256, (3, 3), kernel_initializer='he_normal', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.5))

model.add(Convolution2D(512, (3, 3), kernel_initializer='he_normal', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Convolution2D(512, (3, 3), kernel_initializer='he_normal', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Convolution2D(512, (3, 3), kernel_initializer='he_normal', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.5))

model.add(Convolution2D(512, (3, 3), kernel_initializer='he_normal', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Convolution2D(512, (3, 3), kernel_initializer='he_normal', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Convolution2D(512, (3, 3), kernel_initializer='he_normal', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(7, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=[fbeta, 'accuracy'])
model.summary()

In [ ]:
%%time
history = model.fit_generator(data_generator.flow(x_train, y_train, batch_size),
                    steps_per_epoch=len(x_train) / batch_size,
                    epochs=num_epochs, verbose=True, callbacks=callbacks,
                    validation_data=(x_test, y_test))

In [ ]:
score = model.evaluate(x_validation, y_validation)

print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#  summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('cnn_accuracy.png')
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Plot a confusion matrix
emotions_text = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

y_pred = model.predict_classes(x_validation)
y_true = np.asarray([np.argmax(i) for i in y_validation])

cm = confusion_matrix(y_true, y_pred)
cm_normalised = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.set(font_scale=4.5) 
fig, ax = plt.subplots(figsize=(15,15))
ax = sns.heatmap(cm_normalised, annot=True, linewidths=2.5, square=True, linecolor="Green", 
                    cmap="Greens", yticklabels=emotions_text, xticklabels=emotions_text, vmin=0, vmax=np.max(cm_normalised), 
                    fmt=".2f", annot_kws={"size": 25})
ax.set(xlabel='Predicted label', ylabel='True label')

In [ ]:
beta=1.0
score = fbeta_score(y_true, y_pred, average='micro', beta=beta)
print("fbeta = {}, beta = {}".format(score, beta))

In [ ]:
model_json = model.to_json()
with open("arch_1.json","w") as json_file:
     json_file.write(model_json)
model.save('weights_1.h5')

In [ ]:
# A ESTRATEGIA EH VERIFICAR SE OS ARQUIVOS SALVOS NO CONSOLE PODEM SER SALVOS PARA DOWNLOAD
!ls